In [1]:
import logging
logging.captureWarnings(False)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
from dask.distributed import Client, progress
client = Client(n_workers=5, threads_per_worker=4, memory_limit='30GB')
client

/Users/bkirtman/miniconda3/envs/myenv/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55677 instead
  warnings.warn(
2023-12-29 20:34:05,452 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/jd/v995m27s62746tw3wq1xg1mh0000gn/T/dask-worker-space/worker-fnppkjiw', purging
2023-12-29 20:34:05,452 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/jd/v995m27s62746tw3wq1xg1mh0000gn/T/dask-worker-space/worker-w0t5u9xc', purging
2023-12-29 20:34:05,452 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/jd/v995m27s62746tw3wq1xg1mh0000gn/T/dask-worker-space/worker-opnzr72d', purging
2023-12-29 20:34:05,452 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/jd/v995m27s62746tw3wq1xg1mh0000gn/T/dask-worker-space/worker-yk7d8usi', purging
2023-12-29 2

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55677/status,
Dashboard: http://127.0.0.1:55677/status,Workers: 5
Total threads: 20,Total memory: 139.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55678,Workers: 5
Dashboard: http://127.0.0.1:55677/status,Total threads: 20
Started: Just now,Total memory: 139.70 GiB
Comm: tcp://127.0.0.1:55708,Total threads: 4
Dashboard: http://127.0.0.1:55709/status,Memory: 27.94 GiB
Nanny: tcp://127.0.0.1:55685,


In [2]:
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import scipy as sp
import pandas as pd
from netCDF4 import Dataset
import dask
import torch
import torch.nn as nn
import torch.fft as fft
import subprocess
import sys

from sht_utils import *
from subs1_utils import *

In [3]:
### This cell initializes the model
###
###     First Define all spectral grids
###
zw = 42 # zonal wave number
mw = 42 # meridional wave number
kmax = 11
imax = 128
jmax = 64
steps_per_day = 216 ### Changing this number implies time step changes and should
#                       be implemented carefully
###
spec = (mw,zw,kmax)
grid = (imax,jmax,kmax)
#
#
# provide experiment name and data path for writing out data
# datapath may need to be edited for your system
#
expname = 'PrescribedMeanT42L11F'
foo = str(subprocess.check_output(['whoami']))
end = len(foo) - 3
uname = foo[2:end]
datapath = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/'+expname+'/'
subprocess.call(['rm','-r', datapath])
subprocess.check_output(['mkdir', datapath])

b''

In [4]:
# Get the Gaussian latitudes and equally spaced longitudes
#
cost_lg, wlg = legendre_gauss_weights(jmax, -1, 1)
lats = np.flip(np.arccos(cost_lg))
lats = -90+180*lats/(np.pi)
#
lons = np.linspace(0.0,360.0-360.0/imax,imax)
#
# Initialize grid to spectral (sht) and spectral to grid (isht)
# transforms
#
sht = RealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
isht = InverseRealSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
vsht = RealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)
ivsht = InverseRealVectorSHT(jmax, imax, lmax=mw, mmax=zw, grid="legendre-gauss", csphase=False)

In [5]:
#
# Initialization: Could read spectral restarts, or could
#              start at rest. If wanting to use grid point see
#              jupyter notebook preprocess
#              
#
# Implement at rest initial condition, but need coriolis
#
#
coriolis = np.zeros((jmax,imax))
for jj in range(jmax):
    coriolis[jj,:] = -(4.0*np.pi/86400)*cost_lg[jj] # minus sign because 
                                                    #grid runs South-To-North
#
#
# Initialize spectral fields (at rest or to be read in)
#
zmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn1 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn2 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
qmn1 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn2 = torch.zeros((mw,zw),dtype=torch.complex128)
qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Introduce an initial perturbation to get things going
#
datapath_init = '/Users/'+uname+'/Work/AGCM/AGCM/tmp4/Restarts/PerturbationT42L11/'
zmn1 = torch.load(datapath_init+'zmn1.spectral.pt')
zmn2 = torch.load(datapath_init+'zmn2.spectral.pt')
dmn1 = torch.load(datapath_init+'dmn1.spectral.pt')
dmn2 = torch.load(datapath_init+'dmn2.spectral.pt')
tmn1 = torch.load(datapath_init+'tmn1.spectral.pt')
tmn2 = torch.load(datapath_init+'tmn2.spectral.pt')
qmn1 = torch.load(datapath_init+'qmn1.spectral.pt')
qmn2 = torch.load(datapath_init+'qmn2.spectral.pt')
#
#
# Topography data - this should be spectral data or can be
#                        initialized to zero. If grid point data
#                        is desired see gptosp.agcm.ipynb for how to
#                        convert to spectral. 
#
# Setting topography to zero here
#
phismn = torch.zeros((mw,zw),dtype=torch.complex128)
#
# If non-zero topog read here
#
####phismn = torch.load('topog.spectral.pt') # only read topography is background state is zonally symmetric
#
#
# Adding heating here see preprocess.ipynb
#
heat = torch.load('heat.ggrid.pt')
#
# or set to zero
#
###heat = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
#
#
# Read Climatology on Gausian Grid
#
uclim = torch.load('usig.ggrid.pt')
vclim = torch.load('vsig.ggrid.pt')
tclim = torch.load('tsig.ggrid.pt')
vortclim = torch.load('vortsig.ggrid.pt')
divclim = torch.load('divsig.ggrid.pt')
dxqclim = torch.load('dxq_gg.ggrid.pt')
dyqclim = torch.load('dyq_gg.ggrid.pt')
#
#

In [6]:
###
### Constants, parameters, vertical differencing parameters,
### matricies for geopotential height, etc ...
###
delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)
### The above code is in subs1_utils.py - vertical structure related
### This code would need to be changed if the vertical resolution
### is changed - could be done by simply specifying delsig in bscst
###
amtrx, cmtrx, dmtrx = mcoeff(kmax,si,sl,slkap,r1b,r2b,delsig)
### The above code is for geopotential height and implicit scheme
### in subs1_utils.py but unlikely any changes would be needed
emtrx = inv_em(dmtrx,steps_per_day,kmax,mw,zw)
### The above code
### emtrix is used in the implicit time scheme, computed once here to save cpu time
### changes unlikely

In [7]:
#### Preprocessing is complete - now time to run model
#
#
# The Model Runs in 30-day chuncks - need to specify how many 30-day chunks to run
tl = 30 ##### tl is the chunk size - typically 30 days, but for testing 3 is reasonable
#
# Suggested ichunk for time dependent models: 120
#
ae = 6.371E+06 # Earth radius
tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
#
ddtdiv = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ddtvort = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
junk = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
ttend = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
#
vort = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
div = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
temp = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
qdot = torch.zeros((kmax,jmax,imax),dtype=torch.float64)
times = pd.date_range(start = '1950-01-01', end='2100-01-01', freq='D')
# 
ichunk = 30
#
idays = tl * ichunk
#
#
#
#
# Begin Time Loop
#
ii = 0
savedat = 0
daycount = 0
total_days = 0
nstep = idays*steps_per_day
while ii < nstep:
    ii = ii + 1
    savedat = savedat + 1
    zmnt[daycount] = zmnt[daycount] + zmn1/steps_per_day
    dmnt[daycount] = dmnt[daycount] + dmn1/steps_per_day
    tmnt[daycount] = tmnt[daycount] + tmn1/steps_per_day
    qmnt[daycount] = qmnt[daycount] + qmn1/steps_per_day
    wmnt[daycount] = wmnt[daycount] + wmn1/steps_per_day
    if (savedat == steps_per_day): # post processing
        #
        # Call Postprocessing Routine as needed
        #
        print((dmn1[10,2,2],dmn1[9,2,1]))
        daycount = daycount + 1
        total_days = total_days + 1
        print(['Day = ',total_days])
        if (daycount == tl):
            times_30day = times[total_days-tl:total_days]
            postprocessing(isht,ivsht,zmnt,dmnt,tmnt,qmnt,wmnt,\
                           phismn,amtrx,times_30day,mw,zw,\
                           kmax,imax,jmax,sl,lats,lons,tl,datapath)
            tmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            zmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            dmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            qmnt = torch.zeros((tl,mw,zw),dtype=torch.complex128)
            wmnt = torch.zeros((tl,kmax,mw,zw),dtype=torch.complex128)
            daycount = 0
        savedat = 0
    #
    # Run model for one time step
    #
    # Spectral to grid transformation of needed fields:
    #   Vorticity, divergence, temperature, U, V, 
    #   grad(ln(Ps)), Q prescibed heating
    #
    #
    for k in range(kmax):
        vort[k] = isht(zmn2[k]).cpu() ### This is the relative vorticity
        div[k] = isht(dmn2[k]).cpu()
        temp[k] = isht(tmn2[k]).cpu()
        qdot[k] = isht(wmn2[k]).cpu()
    u,v = uv(ivsht,zmn2,dmn2,mw,zw,kmax,imax,jmax)
    dxq,dyq = gradq(ivsht,qmn2,mw,zw,imax,jmax)
    #
    # Stack grid variables climo first[0] perturbation second[1]
    #
    vort2 = torch.stack((vortclim,vort)) # vortclim is the absolute vorticity
    div2 = torch.stack((divclim,div))
    temp2 = torch.stack((tclim,temp))
    u2 = torch.stack((uclim,u))
    v2 = torch.stack((vclim,v))
    dxq2 = torch.stack((dxqclim,dxq))
    dyq2 = torch.stack((dyqclim,dyq))
    #
    # Non-Linear products
    #
    a,b,e,ut,vt,ri,wj,cbar,dbar = nlprod_prescribed_mean(u2,v2,vort2,div2,temp2,dxq2,dyq2,heat,coriolis,delsig,si,sikap,slkap,\
                                         r1b,r2b,cth1,cth2,cost_lg,kmax,imax,jmax)
    #
    #
    # Grid to spectral transformation of nlprod results
    #
    ddtdiv,ddtvort = vortdivspec(vsht,a,b,kmax,mw,zw)
    zmn3 = - ddtvort
    dmn3 = ddtdiv
    junk,ttend = vortdivspec(vsht,ut,vt,kmax,mw,zw)
    for k in range(kmax):
        dddt = dmn3[k] - lap_sht(sht,e[k],mw,zw) 
        dmn3[k] = dddt
        tmn3[k] = -ttend[k] + sht(ri[k]).cpu()
        wmn3[k] = sht(wj[k]).cpu() ### Prescribed heating converted to spectral
    qmn3 = -sht(cbar[1]).cpu() ### Only cbar here since dbar is included in implicit or explicit
    # 
    # Diffusion, Damping, Implicit or Explicit time differencing, Time filter
    #
    zmn3,dmn3,tmn3 = diffsn(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,\
                            kmax,mw,zw)
    #
    zmn3,dmn3,tmn3,qmn3 = damp_prescribed_mean(zmn1,zmn3,dmn1,dmn3,tmn1,tmn3,qmn1,qmn3,\
                          kmax)
    #
    dt = 86400.0/steps_per_day
    #
    zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,tmn3,qmn1,qmn2,qmn3 = \
                        explicit(dt,amtrx,cmtrx,dmtrx,emtrx,\
                        zmn1,zmn2,zmn3,dmn1,dmn2,dmn3,tmn1,tmn2,\
                        tmn3,wmn1,wmn2,wmn3,qmn1,qmn2,qmn3,phismn,\
                        delsig,kmax,mw,zw)
    ####
    ## Reset zmn3, dmn3, tmn3,wmn3 & qmn3
    ###
    zmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    dmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    tmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    wmn3 = torch.zeros((kmax,mw,zw),dtype=torch.complex128)
    qmn3 = torch.zeros((mw,zw),dtype=torch.complex128)
#
# Done

(tensor(-1.7397e-07-1.1668e-07j, dtype=torch.complex128), tensor(-1.1757e-07+1.8326e-07j, dtype=torch.complex128))
['Day = ', 1]
(tensor(-3.2959e-10+1.0126e-08j, dtype=torch.complex128), tensor(-2.4516e-07+2.9107e-07j, dtype=torch.complex128))
['Day = ', 2]
(tensor(2.0175e-07-8.7955e-08j, dtype=torch.complex128), tensor(-3.6639e-08+8.5444e-08j, dtype=torch.complex128))
['Day = ', 3]
(tensor(2.8687e-07-1.0874e-07j, dtype=torch.complex128), tensor(1.4893e-08+1.0102e-07j, dtype=torch.complex128))
['Day = ', 4]
(tensor(2.4848e-07-1.4224e-07j, dtype=torch.complex128), tensor(-2.2656e-07+1.9145e-07j, dtype=torch.complex128))
['Day = ', 5]
(tensor(1.6312e-07-1.6538e-07j, dtype=torch.complex128), tensor(-1.7997e-07+3.2779e-07j, dtype=torch.complex128))
['Day = ', 6]
(tensor(1.4062e-07-1.3138e-07j, dtype=torch.complex128), tensor(-1.0490e-07-2.4742e-08j, dtype=torch.complex128))
['Day = ', 7]
(tensor(-4.5492e-08+6.8552e-08j, dtype=torch.complex128), tensor(-1.0895e-07-6.6595e-08j, dtype=torch.c

In [8]:
## Write spectral data for possible restart
##
torch.save(zmn1,datapath+'zmn1.spectral.pt')
torch.save(zmn2,datapath+'zmn2.spectral.pt')
torch.save(dmn1,datapath+'dmn1.spectral.pt')
torch.save(dmn2,datapath+'dmn2.spectral.pt')
torch.save(tmn1,datapath+'tmn1.spectral.pt')
torch.save(tmn2,datapath+'tmn2.spectral.pt')
torch.save(qmn1,datapath+'qmn1.spectral.pt')
torch.save(qmn2,datapath+'qmn2.spectral.pt')